In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json
from torchvision.datasets import MNIST, CIFAR10
from torchvision import transforms
import random
import os
import pickle
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Iterator, Tuple, Union, Dict, List
from argparse import ArgumentParser, Namespace
from collections import OrderedDict, Counter
from copy import deepcopy

In [ ]:
x = torch.tensor([1, 2, np.nan, 4])
print(x)
# x!=x
# print(x)
torch.isnan(x)

tensor([1., 2., nan, 4.])


tensor([False, False,  True, False])

In [ ]:
!pip install fedlab~=1.1.4
from fedlab.utils.serialization import SerializationTool
from fedlab.utils.aggregator import Aggregators
from fedlab.utils.dataset.slicing import noniid_slicing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 4.7 MB/s 
  Created wheel for fedlab: filename=fedlab-1.1.5-py3-none-any.whl size=56049 sha256=86ddf05af96f4bead5b8fcf4a5c6823fcd23d551b18bdad37e322c97a1f34223
  Stored in directory: /root/.cache/pip/wheels/d2/fa/bf/095e6b35c4cfcfe0eed549aea1a2b280bc7617f86c6e165810
Successfully built fedlab


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
!pip install rich
import rich
from rich.console import Console
from rich.progress import track

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 8.6 MB/s 


In [ ]:
!pip install path
from path import Path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###MOUNT

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd '/content/drive/Shareddrives/Duong-Binh-BKHN/Simt_perFedAvg'

/content/drive/Shareddrives/Duong-Binh-BKHN/Simt_perFedAvg


In [ ]:
!pwd

/content/drive/Shareddrives/Duong-Binh-BKHN/Simt_perFedAvg


###DATA/DATASET

In [ ]:
class MNISTDataset(Dataset):
    def __init__(
        self,
        subset=None,
        data=None,
        targets=None,
        transform=None,
        target_transform=None,
    ) -> None:
        self.transform = transform
        self.target_transform = target_transform
        if (data is not None) and (targets is not None):
            self.data = data.unsqueeze(1)
            self.targets = targets
        elif subset is not None:
            self.data = torch.stack(
                list(
                    map(
                        lambda tup: tup[0]
                        if isinstance(tup[0], torch.Tensor)
                        else torch.tensor(tup[0]),
                        subset,
                    )
                )
            )
            self.targets = torch.stack(
                list(
                    map(
                        lambda tup: tup[1]
                        if isinstance(tup[1], torch.Tensor)
                        else torch.tensor(tup[1]),
                        subset,
                    )
                )
            )
        else:
            raise ValueError(
                "Data Format: subset: Tuple(data: Tensor / Image / np.ndarray, targets: Tensor) OR data: List[Tensor]  targets: List[Tensor]"
            )

    def __getitem__(self, index):
        data, targets = self.data[index], self.targets[index]

        if self.transform is not None:
            data = self.transform(self.data[index])

        if self.target_transform is not None:
            targets = self.target_transform(self.targets[index])

        return data, targets

    def __len__(self):
        return len(self.targets)


class CIFARDataset(Dataset):
    def __init__(
        self,
        subset=None,
        data=None,
        targets=None,
        transform=None,
        target_transform=None,
    ) -> None:
        self.transform = transform
        self.target_transform = target_transform
        if (data is not None) and (targets is not None):
            self.data = data.unsqueeze(1)
            self.targets = targets
        elif subset is not None:
            self.data = torch.stack(
                list(
                    map(
                        lambda tup: tup[0]
                        if isinstance(tup[0], torch.Tensor)
                        else torch.tensor(tup[0]),
                        subset,
                    )
                )
            )
            self.targets = torch.stack(
                list(
                    map(
                        lambda tup: tup[1]
                        if isinstance(tup[1], torch.Tensor)
                        else torch.tensor(tup[1]),
                        subset,
                    )
                )
            )
        else:
            raise ValueError(
                "Data Format: subset: Tuple(data: Tensor / Image / np.ndarray, targets: Tensor) OR data: List[Tensor]  targets: List[Tensor]"
            )

    def __getitem__(self, index):
        img, targets = self.data[index], self.targets[index]

        if self.transform is not None:
            img = self.transform(self.data[index])

        if self.target_transform is not None:
            targets = self.target_transform(self.targets[index])

        return img, targets

    def __len__(self):
        return len(self.targets)


###PREPROCESS

In [ ]:
#CURRENT_DIR = Path(__file__).parent.abspath()
CURRENT_DIR = "/content/drive/Shareddrives/Duong-Binh-BKHN/Simt_perFedAvg"

DATASET = {
    "mnist": (MNIST, MNISTDataset),
    "cifar": (CIFAR10, CIFARDataset),
}

MEAN = {
    "mnist": (0.1307,),
    "cifar": (0.4914, 0.4822, 0.4465),
}

STD = {
    "mnist": (0.3015,),
    "cifar": (0.2023, 0.1994, 0.2010),
}

def preprocess(args: Namespace) -> None:
    dataset_dir = CURRENT_DIR + "/" + args.dataset
    pickles_dir = CURRENT_DIR + "/" + args.dataset + "/" + "pickles"

    np.random.seed(args.seed)
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    num_train_clients = int(args.client_num_in_total * args.fraction)
    num_test_clients = args.client_num_in_total - num_train_clients

    transform = transforms.Compose(
        [transforms.Normalize(MEAN[args.dataset], STD[args.dataset]),]
    )
    target_transform = None
    trainset_stats = {}
    testset_stats = {}

    if not os.path.isdir(CURRENT_DIR + "/" + args.dataset):
        os.mkdir(CURRENT_DIR + "/" + args.dataset)
    if os.path.isdir(pickles_dir):
        os.system(f"rm -rf {pickles_dir}")
    os.mkdir(f"{pickles_dir}")

    ori_dataset, target_dataset = DATASET[args.dataset]
    trainset = ori_dataset(
        dataset_dir, train=True, download=True, transform=transforms.ToTensor()
    )
    testset = ori_dataset(dataset_dir, train=False, transform=transforms.ToTensor())

    num_classes = 10 if args.classes <= 0 else args.classes
    all_trainsets, trainset_stats = randomly_alloc_classes(
        ori_dataset=trainset,
        target_dataset=target_dataset,
        num_clients=num_train_clients,
        num_classes=num_classes,
        transform=transform,
        target_transform=target_transform,
    )
    all_testsets, testset_stats = randomly_alloc_classes(
        ori_dataset=testset,
        target_dataset=target_dataset,
        num_clients=num_test_clients,
        num_classes=num_classes,
        transform=transform,
        target_transform=target_transform,
    )

    all_datasets = all_trainsets + all_testsets

    for client_id, dataset in enumerate(all_datasets):
        with open(pickles_dir + "/" + str(client_id) + ".pkl", "wb") as f:
            pickle.dump(dataset, f)
    with open(pickles_dir + "/" + "seperation.pkl", "wb") as f:
        pickle.dump(
            {
                "train": [i for i in range(num_train_clients)],
                "test": [i for i in range(num_train_clients, args.client_num_in_total)],
                "total": args.client_num_in_total,
            },
            f,
        )
    with open(dataset_dir + "/" + "all_stats.json", "w") as f:
        json.dump({"train": trainset_stats, "test": testset_stats}, f)

def randomly_alloc_classes(
    ori_dataset: Dataset,
    target_dataset: Dataset,
    num_clients: int,
    num_classes: int,
    transform=None,
    target_transform=None,
) -> Tuple[List[Dataset], Dict[str, Dict[str, int]]]:
    dict_users = noniid_slicing(ori_dataset, num_clients, num_clients * num_classes)
    stats = {}
    for i, indices in dict_users.items():
        targets_numpy = np.array(ori_dataset.targets)
        stats[f"client {i}"] = {"x": 0, "y": {}}
        stats[f"client {i}"]["x"] = len(indices)
        stats[f"client {i}"]["y"] = Counter(targets_numpy[indices].tolist())
    datasets = []
    for indices in dict_users.values():
        datasets.append(
            target_dataset(
                [ori_dataset[i] for i in indices],
                transform=transform,
                target_transform=target_transform,
            )
        )
    return datasets, stats

class get_args_preprocess():
  def __init__(self):
    self.dataset = "mnist"
    self.client_num_in_total = 200
    self.fraction = 0.9
    self.classes = 2
    self.seed = 0

args = get_args_preprocess()
preprocess(args)

/usr/local/lib/python3.8/dist-packages/fedlab/utils/dataset/slicing.py:38: UserWarning: warning: the length of dataset isn't divided exactly by num_shard.some samples will be dropped.
  warnings.warn(


###DATA/UTILS

In [ ]:
DATASET_DICT = {
    "mnist": MNISTDataset,
    "cifar": CIFARDataset,
}
#CURRENT_DIR = Path(__file__).parent.abspath()
CURRENT_DIR = "/content/drive/Shareddrives/Duong-Binh-BKHN/Simt_perFedAvg"

def get_dataloader(dataset: str, client_id: int, batch_size=20, valset_ratio=0.1):
    pickles_dir = CURRENT_DIR + "/" + dataset + "/" "pickles"
    if os.path.isdir(pickles_dir) is False:
        raise RuntimeError("Please preprocess and create pickles first.")

    with open(pickles_dir + "/" + str(client_id) + ".pkl", "rb") as f:
        client_dataset: DATASET_DICT[dataset] = pickle.load(f)

    val_num_samples = int(valset_ratio * len(client_dataset))
    train_num_samples = len(client_dataset) - val_num_samples

    trainset, valset = random_split(
        client_dataset, [train_num_samples, val_num_samples]
    )
    trainloader = DataLoader(trainset, batch_size, drop_last=True)
    valloader = DataLoader(valset, batch_size)

    return trainloader, valloader

def get_client_id_indices(dataset):
    dataset_pickles_path = CURRENT_DIR + "/" + dataset + "/" + "pickles"
    with open(dataset_pickles_path + "/" + "seperation.pkl", "rb") as f:
        seperation = pickle.load(f)
    return (seperation["train"], seperation["test"], seperation["total"])

###MODEL

In [ ]:
class elu(nn.Module):
    def __init__(self) -> None:
        super(elu, self).__init__()

    def forward(self, x):
        return torch.where(x >= 0, x, 0.2 * (torch.exp(x) - 1))


class linear(nn.Module):
    def __init__(self, in_c, out_c) -> None:
        super(linear, self).__init__()
        self.w = nn.Parameter(
            torch.randn(out_c, in_c) * torch.sqrt(torch.tensor(2 / in_c))
        )
        self.b = nn.Parameter(torch.randn(out_c))

    def forward(self, x):
        return F.linear(x, self.w, self.b)


class MLP_MNIST(nn.Module):
    def __init__(self) -> None:
        super(MLP_MNIST, self).__init__()
        self.fc1 = linear(28 * 28, 80)
        self.fc2 = linear(80, 60)
        self.fc3 = linear(60, 10)
        self.flatten = nn.Flatten()
        self.activation = elu()

    def forward(self, x):
        x = self.flatten(x)

        x = self.fc1(x)
        x = self.activation(x)

        x = self.fc2(x)
        x = self.activation(x)

        x = self.fc3(x)
        x = self.activation(x)

        return x


class MLP_CIFAR10(nn.Module):
    def __init__(self) -> None:
        super(MLP_CIFAR10, self).__init__()
        self.fc1 = linear(32 * 32 * 3, 80)
        self.fc2 = linear(80, 60)
        self.fc3 = linear(60, 10)
        self.flatten = nn.Flatten()
        self.activation = elu()

    def forward(self, x):
        x = self.flatten(x)

        x = self.fc1(x)
        x = self.activation(x)

        x = self.fc2(x)
        x = self.activation(x)

        x = self.fc3(x)
        x = self.activation(x)

        return x

MODEL_DICT = {"mnist": MLP_MNIST, "cifar": MLP_CIFAR10}

def get_model(dataset, device):
    return MODEL_DICT[dataset]().to(device)

###UTILS

In [ ]:
class get_args():
  def __init__(self):
    self.alpha = 1e-2
    self.beta = 1e-3
    self.lmda = 0.2
    self.global_epochs = 200
    self.local_epochs = 4
    self.pers_epochs = 1
    self.hf = 1
    self.batch_size = 40
    self.valset_ratio = 0.1
    self.dataset = "mnist"  #choices = ["mnist", "cifar"]
    self.client_num_per_round = 10
    self.seed = 17
    self.gpu = 1
    self.eval_while_training = 1
    self.log = 0
    
# def get_args():
#     parser = ArgumentParser()
#     parser.add_argument("--alpha", type=float, default=1e-2)
#     parser.add_argument("--beta", type=float, default=1e-3)
#     parser.add_argument("--global_epochs", type=int, default=200)
#     parser.add_argument("--local_epochs", type=int, default=4)
#     parser.add_argument(
#         "--pers_epochs",
#         type=int,
#         default=1,
#         help="Indicate how many data batches would be used for personalization. Negatives means that equal to train phase.",
#     )
#     parser.add_argument(
#         "--hf",
#         type=int,
#         default=1,
#         help="0 for performing Per-FedAvg(FO), others for Per-FedAvg(HF)",
#     )
#     parser.add_argument("--batch_size", type=int, default=40)
#     parser.add_argument(
#         "--valset_ratio",
#         type=float,
#         default=0.1,
#         help="Proportion of val set in the entire client local dataset",
#     )
#     parser.add_argument(
#         "--dataset", type=str, choices=["mnist", "cifar"], default="mnist"
#     )
#     parser.add_argument("--client_num_per_round", type=int, default=10)
#     parser.add_argument("--seed", type=int, default=17)
#     parser.add_argument(
#         "--gpu",
#         type=int,
#         default=1,
#         help="Non-zero value for using gpu, 0 for using cpu",
#     )
#     parser.add_argument(
#         "--eval_while_training",
#         type=int,
#         default=1,
#         help="Non-zero value for performing local evaluation before and after local training",
#     )
#     parser.add_argument("--log", type=int, default=0)
#     return parser.parse_args()


@torch.no_grad()
def eval(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: Union[torch.nn.MSELoss, torch.nn.CrossEntropyLoss],
    device=torch.device("cpu"),
) -> Tuple[torch.Tensor, torch.Tensor]:
    model.eval()
    total_loss = 0
    num_samples = 0
    acc = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        logit = model(x)
        # total_loss += criterion(logit, y) / y.size(-1)
        total_loss += criterion(logit, y)
        pred = torch.softmax(logit, -1).argmax(-1)
        acc += torch.eq(pred, y).int().sum()
        num_samples += y.size(-1)
    model.train()
    return total_loss, acc / num_samples

def get_data_batch(
    dataloader: torch.utils.data.DataLoader,
    iterator: Iterator,
    device=torch.device("cpu"),
):
    try:
        x, y = next(iterator)
    except StopIteration:
        iterator = iter(dataloader)
        x, y = next(iterator)

    return x.to(device), y.to(device)

def fix_random_seed(seed: int):
    torch.cuda.empty_cache()
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

###PERFEDAVG

In [ ]:
class PerFedAvgClient:
    def __init__(
        self,
        client_id: int,
        alpha: float,
        beta: float,
        lmbda: float,
        global_model: torch.nn.Module,
        momentum_model: torch.nn.Module,
        criterion: Union[torch.nn.CrossEntropyLoss, torch.nn.MSELoss],
        batch_size: int,
        dataset: str,
        local_epochs: int,
        valset_ratio: float,
        logger: rich.console.Console,
        gpu: int,
    ):

        # Set cuda 
        if gpu and torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.logger = logger

        self.local_epochs = local_epochs
        self.criterion = criterion
        self.id = client_id

        self.model = deepcopy(global_model)
        self.momentum_model = deepcopy(momentum_model)

        self.alpha = alpha
        self.beta = beta
        self.lamda = lmbda
        self.trainloader, self.valloader = get_dataloader(
            dataset, client_id, batch_size, valset_ratio
        )
        self.iter_trainloader = iter(self.trainloader)

    def train(
        self,
        global_model: torch.nn.Module,
        hessian_free=False,
        eval_while_training=False,
    ):
        self.model.load_state_dict(global_model.state_dict())
        if eval_while_training:
            loss_before, acc_before = eval(
                self.model, self.valloader, self.criterion, self.device
            )
        self._train(hessian_free)

        if eval_while_training:
            loss_after, acc_after = eval(
                self.model, self.valloader, self.criterion, self.device
            )
            self.logger.log(
                "client [{}] [red]loss: {:.4f} -> {:.4f}   [blue]acc: {:.2f}% -> {:.2f}%".format(
                    self.id,
                    loss_before,
                    loss_after,
                    acc_before * 100.0,
                    acc_after * 100.0,
                )
            )

        # Return model parameters
        return SerializationTool.serialize_model(self.model)

    def _train(self, hessian_free=False):
        if hessian_free:  # Per-FedAvg(HF)
            for _ in range(self.local_epochs):
              
                temp_model = deepcopy(self.model)
                temp_momentum_model = deepcopy(self.momentum_model)
            
                data_batch_1 = get_data_batch(
                    self.trainloader, self.iter_trainloader, self.device
                )
                grads = self.compute_grad(temp_model, temp_momentum_model, data_batch_1)                # adapt (global theta, data_batch_1).           
                for param, grad in zip(temp_model.parameters(), grads):
                    param.data.sub_(self.alpha * grad) 

                # data_batch_1_ = get_data_batch(
                #     self.trainloader, self.iter_trainloader, self.device
                # )
                grads_ = self.compute_grad(temp_momentum_model, temp_momentum_model, data_batch_1)      # adapt (momentum theta, data_batch_1_)        
                for param_, grad_ in zip(temp_momentum_model.parameters(), grads_):
                    param_.data.sub_(self.alpha * grad_)                          

                # Dropout (theta_global model)
                # phi_dropout_model=self.Dropout_func(temp_model,0.2)
                phi_params = SerializationTool.serialize_model(temp_model)
                m = nn.Dropout(p=0.2)
                phi_params_output = m(phi_params)

                phi_dropout_model = deepcopy(self.model)

                SerializationTool.deserialize_model(phi_dropout_model, phi_params_output)

                data_batch_2 = get_data_batch(
                    self.trainloader, self.iter_trainloader, self.device
                )
                grads_1st = self.compute_grad(phi_dropout_model, temp_momentum_model, data_batch_2)
                   
                loss_teach = self.compute_grad(phi_dropout_model, temp_momentum_model, data_batch_2, KL_div=True)

                data_batch_3 = get_data_batch(
                    self.trainloader, self.iter_trainloader, self.device
                )

                grads_2nd = self.compute_grad(
                    self.model, temp_momentum_model, data_batch_3, v=grads_1st, second_order_grads=True
                )
                # NOTE: Go check https://github.com/KarhouTam/Per-FedAvg/issues/2 if you confuse about the model update.
                for param, grad2, lossteach in zip(
                    self.model.parameters(), grads_2nd, loss_teach      # grad_total
                ):
                    param.data.sub_(self.beta * lossteach - self.beta * self.alpha * grad2)

                # for param, grad1, grad2, grad_loss_teach in zip(
                #     self.model.parameters(), grads_1st, grads_2nd, loss_teach          # grad_total
                # ):
                #     param.data.sub_(self.beta * grad1 - self.beta * self.alpha * grad2)

        # else:  # Per-FedAvg(FO)
        #     for _ in range(self.local_epochs):
        #         # ========================== FedAvg ==========================
        #         # NOTE: You can uncomment those codes for running FedAvg.
        #         #       When you're trying to run FedAvg, comment other codes in this branch.

        #         # data_batch = utils.get_data_batch(
        #         #     self.trainloader, self.iter_trainloader, self.device
        #         # )
        #         # grads = self.compute_grad(self.model, data_batch)
        #         # for param, grad in zip(self.model.parameters(), grads):
        #         #     param.data.sub_(self.beta * grad)

        #         # ============================================================

        #         temp_model = deepcopy(self.model)
        #         data_batch_1 = get_data_batch(
        #             self.trainloader, self.iter_trainloader, self.device
        #         )
        #         grads = self.compute_grad(temp_model, data_batch_1)

        #         for param, grad in zip(temp_model.parameters(), grads):
        #             param.data.sub_(self.alpha * grad)

        #         data_batch_2 = get_data_batch(
        #             self.trainloader, self.iter_trainloader, self.device
        #         )
        #         grads = self.compute_grad(temp_model, data_batch_2)

        #         for param, grad in zip(self.model.parameters(), grads):
        #             param.data.sub_(self.beta * grad)

    def compute_grad(
        self,
        model: torch.nn.Module,
        teacher_model: torch.nn.Module,
        data_batch: Tuple[torch.Tensor, torch.Tensor],
        v: Union[Tuple[torch.Tensor, ...], None] = None,
        second_order_grads=False,
        KL_div=False,
    ):
        x, y = data_batch
        if second_order_grads:
            frz_model_params = deepcopy(model.state_dict())
            delta = 1e-3
            dummy_model_params_1 = OrderedDict()
            dummy_model_params_2 = OrderedDict()
            with torch.no_grad():
                for (layer_name, param), grad in zip(model.named_parameters(), v):
                    dummy_model_params_1.update({layer_name: param + delta * grad})
                    dummy_model_params_2.update({layer_name: param - delta * grad})

            model.load_state_dict(dummy_model_params_1, strict=False)
            logit_1 = model(x)
            # loss_1 = self.criterion(logit_1, y) / y.size(-1)
            loss_1 = self.criterion(logit_1, y)
            grads_1 = torch.autograd.grad(loss_1, model.parameters())

            model.load_state_dict(dummy_model_params_2, strict=False)
            logit_2 = model(x)
            loss_2 = self.criterion(logit_2, y)
            # loss_2 = self.criterion(logit_2, y) / y.size(-1)
            grads_2 = torch.autograd.grad(loss_2, model.parameters())

            model.load_state_dict(frz_model_params)

            grads = []
            with torch.no_grad():
                for g1, g2 in zip(grads_1, grads_2):
                    grads.append((g1 - g2) / (2 * delta))
            return grads

        elif KL_div:
            output_batch = model(x)
            label_batch = y
            output_teacher_batch = teacher_model(x)
            params_temp = 4

            KL_loss = nn.KLDivLoss(reduction="batchmean")(F.log_softmax(output_batch/params_temp, dim=1),
                      F.softmax(output_teacher_batch/params_temp, dim=1)) * (params_temp * params_temp) * (1 - self.lamda) + \
                      F.cross_entropy(output_batch, label_batch) * self.lamda                                                 

            grads = torch.autograd.grad(KL_loss, model.parameters())
            
            return grads

        else:
            logit = model(x)
            # loss = self.criterion(logit, y) / y.size(-1)
            loss = self.criterion(logit, y)
            grads = torch.autograd.grad(loss, model.parameters())
            
            return grads



    def pers_N_eval(self, global_model: torch.nn.Module, pers_epochs: int):
        self.model.load_state_dict(global_model.state_dict())

        loss_before, acc_before = eval(
            self.model, self.valloader, self.criterion, self.device
        )
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.alpha)
        for _ in range(pers_epochs):
            x, y = get_data_batch(
                self.trainloader, self.iter_trainloader, self.device
            )
            logit = self.model(x)
            # loss = self.criterion(logit, y) / y.size(-1)
            loss = self.criterion(logit, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_after, acc_after = eval(
            self.model, self.valloader, self.criterion, self.device
        )
        self.logger.log(
            "client [{}] [red]loss: {:.4f} -> {:.4f}   [blue]acc: {:.2f}% -> {:.2f}%".format(
                self.id, loss_before, loss_after, acc_before * 100.0, acc_after * 100.0,
            )
        )
        return {
            "loss_before": loss_before,
            "acc_before": acc_before,
            "loss_after": loss_after,
            "acc_after": acc_after,
        }

###MAIN

In [ ]:
if __name__ == "__main__":
    args = get_args()
    fix_random_seed(args.seed)
    if os.path.isdir("./log") == False:
        os.mkdir("./log")
    if args.gpu and torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    global_model = get_model(args.dataset, device)
    momentum_model= deepcopy(global_model)
    logger = Console(record=args.log)
    #logger.log(f"Arguments:", dict(args._get_kwargs()))
    clients_4_training, clients_4_eval, client_num_in_total = get_client_id_indices(
        args.dataset
    )

    # init clients
    clients = [
        PerFedAvgClient(
            client_id=client_id,
            alpha=args.alpha,
            beta=args.beta,
            lmbda=args.lmda,
            global_model=global_model,
            momentum_model=momentum_model,
            criterion=torch.nn.CrossEntropyLoss(),
            batch_size=args.batch_size,
            dataset=args.dataset,
            local_epochs=args.local_epochs,
            valset_ratio=args.valset_ratio,
            logger=logger,
            gpu=args.gpu,
        )
        for client_id in range(client_num_in_total)
    ]
    # training
    #logger.log("=" * 20, "TRAINING", "=" * 20, style="bold red")
    for _ in track(
        range(args.global_epochs), "Training...", console=logger, disable=args.log
    ):
        # select clients
        selected_clients = random.sample(clients_4_training, args.client_num_per_round)

        model_params_cache = []
        # client local training
        for client_id in selected_clients:
            serialized_model_params = clients[client_id].train(
                global_model=global_model,
                hessian_free=args.hf,
                eval_while_training=args.eval_while_training,
            )
            model_params_cache.append(serialized_model_params)

        # aggregate model parameters
        aggregated_model_params = Aggregators.fedavg_aggregate(model_params_cache)
        momemtum_params = SerializationTool.serialize_model(momentum_model)
        momemtum_params = 0.995 * momemtum_params + (1-0.995) * aggregated_model_params
        
        SerializationTool.deserialize_model(global_model, aggregated_model_params)
        SerializationTool.deserialize_model(momentum_model, momemtum_params)
        #assign parameters to self.model
        #logger.log("=" * 60)
    # eval
    pers_epochs = args.local_epochs if args.pers_epochs == -1 else args.pers_epochs
    #logger.log("=" * 20, "EVALUATION", "=" * 20, style="bold blue")
    loss_before = []
    loss_after = []
    acc_before = []
    acc_after = []
    for client_id in track(
        clients_4_eval, "Evaluating...", console=logger, disable=args.log
    ):
        stats = clients[client_id].pers_N_eval(
            global_model=global_model, pers_epochs=pers_epochs,               #PerFedAvg
        )
        loss_before.append(stats["loss_before"])
        loss_after.append(stats["loss_after"])
        acc_before.append(stats["acc_before"])
        acc_after.append(stats["acc_after"])

    #logger.log("=" * 20, "RESULTS", "=" * 20, style="bold green")
    #logger.log(f"loss_before_pers: {(sum(loss_before) / len(loss_before)):.4f}")
    #logger.log(f"acc_before_pers: {(sum(acc_before) * 100.0 / len(acc_before)):.2f}%")
    #logger.log(f"loss_after_pers: {(sum(loss_after) / len(loss_after)):.4f}")
    #logger.log(f"acc_after_pers: {(sum(acc_after) * 100.0 / len(acc_after)):.2f}%")

    if args.log:
        algo = "HF" if args.hf else "FO"
        # logger.save_html(
        #     f"./log/{args.dataset}_{args.client_num_per_round}_{args.global_epochs}_{pers_epochs}_{algo}.html"
        # )